# Welome to Tech Day NLP Demonstration

In [1]:
import pandas as pd
import numpy as np
import time
import glob
import os

pd.options.display.max_columns = None

In [8]:
###
#    The first function
#
#    This function reads in the data from the csv file
#    It then processes the data into separate dataframes
#    These dataframes are contained within a dictionary
#    The function returns the dictionary of dataframes
#    for further processing
###

def read_data():
    df = pd.read_csv('./file.csv')
    
    # Initialize an empty dictionary to become the dictionary of dataframes
    d = {}    

    # rename the columns from crap into good stuff based on actual column names from csv file
    columns_good = list(df.iloc[6])
    columns_bad = list(df.columns)
    cols_dict = dict(zip(columns_bad, columns_good))
    
    df.rename(columns=cols_dict, inplace=True)

    # Sanity check printing
    # print(len(columns_good))
    # print(len(columns_bad))
    # cols_dict
    
    # Find all unique sources dropping garbage at top of CSV file
    source_list = list(df.iloc[:,1].unique())[6:]    

    # Sanity check
    # print(source_list)    

    # create a list of the columsn that contain date/time information (used in the for loop below)
    dt_cols = ['Date(ET)', 'Time(ET)', 'LocalTime']
    
    # Create a for loop to build a dictionary of dataframes
    for i in source_list:
        d['{0}'.format(i)] = df[df.iloc[:,1] == i]
        d['{0}'.format(i)].reset_index(inplace=True, drop=True)
        
        # Fix the datetime dtype issue.
        # This applies the to_datetime function to the three identified rows that contain date/time data
        # 
        # IMPORTANT
        # Because all sources were merged into ONE dataframe, some info in the date/time columns was NOT
        # date/time info. For this reason, we use 'errors='ignore''. This keeps the original data that was
        # NOT date/time data intact for when we split the data out into individual dataframes below
        for col in dt_cols:
            d['{0}'.format(i)][col] = pd.to_datetime(d['{0}'.format(i)][col], errors='ignore')
    # for key in d.keys():
        # print(key)
        # d[key].iloc[:, 4:7] = d[key].iloc[:, 4:7].apply(pd.to_datetime, errors='ignore')

    
    # Create a for loop to drop columns that are completely NaN in EACH dataframe
    # Also drop any rows that are completely NaN in EACH dataframe
    for j in source_list:
    #    print(d[j].shape)    # Sanity Check
        d[j].dropna(axis=1, how='all', inplace=True)
    #    print(d[j].shape)    # Sanity Check
        d[j].dropna(axis=0, how='all', inplace=True)
    #    print(d[j].shape)    # Sanity Check
        d[j].reset_index(inplace=True, drop=True)
        
    # Return the dictionary of dataframes 
    return(d)

In [9]:
start = time.time()
df = read_data()
print(df.keys())
end = time.time()
print(end - start)

C:\Users\hobor\Anaconda3\lib\site-packages\ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\hobor\Anaconda3\lib\site-packages\ipykernel_launcher.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\hobor\Anaconda3\lib\site-packages\ipykernel_launcher.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


dict_keys(['NEWS', 'FORUMS', 'TUMBLR', 'TWITTER', 'INSTAGRAM'])
11.759976863861084


In [ ]:
for key in df.keys():
    print(df[key].dtypes)